In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import davies_bouldin_score, silhouette_score, calinski_harabasz_score
from sklearn.mixture import GaussianMixture 
from sklearn.cluster import KMeans, DBSCAN, MeanShift, estimate_bandwidth
import itertools
from tqdm import tqdm


: 

## Clustering of Merchants
### **Objective**: Perform a hard clustering on the merchants to obtain 3 to 5 clusters, which can be used for market segmentation.
### **Overview**: In this notebook, we aim to uncover some underlying cluster structure of our candidate merchant based on three engineered features (as listed below). These features are essential metrics for quantifying the business scale of a merchant, as we believe that not only the merchants with high volume of orders and customers but also small merchants should be given an opportunity to enlarge their business through availability of BNPL service for customers.
### **Attributes for clustering**:
   1. total_number_of_distinct_customers:
   2. monthly_average_number_of_orders:
   3. monthly_average_bnpl_revenue:
### **Clustering pipeline**:
 0. Preliminary Data Analysis
 1. Data Clustering
    1. K-means
    2. MeanShift
    3. DBSCAN
    4. Gaussian Mixture Model
 2. Clustering Evaluation
    1. Silhouette Coefficient
    2. Calinski-Harabasz Index
    3. Davies-Bouldin Index
 3. Clustering Model Deployment


In [ ]:
merchant_df = pd.read_csv("../data/curated/clusters/input/agg_transaction_train.csv")
merchant_df = merchant_df[["merchant_abn",
                           "total_number_of_distinct_customers",
                           "monthly_average_number_of_orders",
                           "monthly_average_bnpl_revenue",
                           "take_rate"
                           ]]
merchant_df.head(5)

: 

### **PDA**
#### 1. The binary correlations between the three attributes are good, no multi-collinearity exists.
#### 2. As we can see that the distributions of the three attributes are very right-skewed, which can potentially lead to clusters that are too uneven in size. Therefore, log transformations will be beneficial to unskew the data.

In [ ]:
features = [
            "monthly_average_bnpl_revenue",
            "monthly_average_number_of_orders",
            "total_number_of_distinct_customers"
            ]

X = merchant_df[features]

X.corr()


: 

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (12, 4), dpi = 100)
for i in range(len(features)):
    sns.histplot(merchant_df[features[i]], 
                stat = 'probability', 
                bins=50,
                ax = ax[i])


# It seems that we can perform GMM on log(features)
fig, ax = plt.subplots(1, 3, figsize = (14, 4), dpi = 100)
for i in range(len(features)):
    sns.histplot(merchant_df[features[i]].apply(np.log), 
                stat = 'probability', 
                bins=50,
                ax = ax[i])
    ax[i].set_xlabel(f'log({ax[i].get_xlabel()})')

: 

### **Clustering**

In [ ]:
log_X = X.apply(np.log)

def homemade_GridSearch(model, parameters, data):
    """
    return a dataframe showing the gridsearch result
    parameters:
               model: a sklearn model
               parameters: a dictionary of parameters for the model, each parameter is a list of possible value
               data: the dataset to fit
    return:
           a dataframe which shows the performance of all combinations of parameters
    """
    # Name of the parameters
    parameter_names = list(parameters.keys())
    # Evaluation metrics
    eval_metrics = [silhouette_score, davies_bouldin_score, calinski_harabasz_score]
    # Name of the evaluation metrics
    eval_metric_names = ['silhouette_score', 'davies_bouldin_score', 'calinski_harabasz_score']

    # Store the performance in a dict
    performance_dict = {parameter: [] for parameter in parameter_names + eval_metric_names + ["num_clusters"]}

    for parameter_value in tqdm(itertools.product(*map(parameters.get, list(parameter_names)))):
        # Store the combination of parameters in a dict
        parameter_set = dict(zip(parameter_names, parameter_value))
        labels = model.set_params(**parameter_set).fit_predict(data)
        # Store the parameters and corresponding score
        for parameter_name in parameter_names:
            performance_dict[parameter_name].append(parameter_set[parameter_name])
        for eval_metric_name, eval_metric in zip(eval_metric_names, eval_metrics):
            try:
                score = eval_metric(data, labels)
            except:
                # If score cannot be calculated, return NaN
                score = np.nan
            performance_dict[eval_metric_name].append(score)
       
        # Add an additional column indicating the number of clusters
        performance_dict["num_clusters"].append(len(np.unique(labels)))

    return pd.DataFrame(performance_dict)
       


: 

### K-Means

In [ ]:
k_mean_params = {
    'n_clusters': [3, 4, 5],
    'random_state': [2022, ]
}

display(homemade_GridSearch(KMeans(), k_mean_params, X))
homemade_GridSearch(KMeans(), k_mean_params, log_X)

: 

### MeanShift

In [ ]:
mean_shift_params1 = {
     'bandwidth': [estimate_bandwidth(X, quantile=q) for q in np.linspace(0.00099, 0.9999, num=10)]
     }

mean_shift_params2 = {
     'bandwidth': [estimate_bandwidth(log_X, quantile=q) for q in np.linspace(0.00099, 0.9999, num=10)]
     }

display(homemade_GridSearch(MeanShift(), mean_shift_params1, X).query("num_clusters <= 5 & num_clusters >= 3"))
homemade_GridSearch(MeanShift(), mean_shift_params2, log_X).query("num_clusters <= 5 & num_clusters >= 3")

: 

### DBSCAN

In [ ]:
dbscan_params1 = {
    'eps': [10, 100, 1000, 5000, 10000, 50000, 100000],
    'min_samples': [1, 5, 10, 20, 50, 100, 500, 1000]
}
dbscan_params2 = {
    'eps': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'min_samples': [1, 5, 10, 20, 50, 100, 500, 1000]
}

display(homemade_GridSearch(DBSCAN(), dbscan_params1, X).query("num_clusters <= 5 & num_clusters >= 3"))
homemade_GridSearch(DBSCAN(), dbscan_params2, log_X).query("num_clusters <= 5 & num_clusters >= 3")

: 

### Gaussian Mixture Model

In [ ]:
gmm_params = {
    'n_components': [3, 4, 5],
    'init_params': ['kmeans', 'random'],
    'random_state': [2022, ]
}

display(homemade_GridSearch(GaussianMixture(), gmm_params, X))
homemade_GridSearch(GaussianMixture(), gmm_params, log_X)

: 

### Cluster Visualization

In [ ]:
%matplotlib ipympl

def visualize_cluster(model, data, figname):
    """
    return a figure of clusters
    """
    data_copy = data.copy()

    # Define X, Y and Z axes
    xs = data_copy[features[0]]
    ys = data_copy[features[1]]
    zs = data_copy[features[2]]
    
    # Clusters of the data
    data_copy["cluster"] = model.fit_predict(data_copy)

    # Create a figure
    figure = plt.figure(dpi = 100)
    ax = figure.add_subplot(projection='3d')
    ax.scatter(xs, ys, zs, c=data_copy["cluster"])

    ax.set_title(figname)
    ax.set_xlabel(features[0])
    ax.set_ylabel(features[1])
    ax.set_zlabel(features[2])
    
    return figure

fg1 = visualize_cluster(KMeans(n_clusters=3), log_X, "Kmean Clustering of logX")
fg2 = visualize_cluster(GaussianMixture(n_components=3, init_params='kmeans'), log_X, "GMM Clustering of logX")
fg3 = visualize_cluster(DBSCAN(eps=10000, min_samples=10), X, "DBSCAN Clustering of X")
fg4 = visualize_cluster(MeanShift(bandwidth=1.984250), log_X, "MeanShift Clustering of logX")

: 

In [ ]:
# Final clustering model
gmm = GaussianMixture(n_components=3, init_params='kmeans').fit(X)
merchant_df["label"] = gmm.predict(X)

# Predict the cluster of the unknown merchants 
pred = pd.read_csv("../data/curated/clusters/input/agg_transaction_pred.csv")
pred["label"] = gmm.predict(pred[features].apply(np.log))
pd.concat([merchant_df[["merchant_abn", "label"]], pred[["merchant_abn", "label"]]]) \
  .to_csv("../data/curated/clusters/output/merchant_clusters.csv", index=False)


# Fill missing take rate with cluster mean
merchant_df.groupby("label", as_index=False)["take_rate"].mean().to_csv("../data/curated/clusters/output/take_rate.csv", index=False)

: 